# Dynamic Nelson-Siegel Model
Xutao Chen

DNS (Dynamic Nelson-Siegel) Model is a classical parameteric term-structure model for fixed income. This Notebook will show how to constrcut DNS and tune its parameter. Then I will use it to do prediction and compare its result with random walk.

In [1]:
#Import all necessary package
from scipy.optimize import least_squares, leastsq
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.arima_model import ARMA
from datetime import timedelta
import holidays
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
#Load the data -- Constant maturity yield in US market
df = pd.read_csv("CMT_Rates.csv")
df.set_index(['Date'],inplace=True)
df.index = pd.to_datetime(df.index,yearfirst=True)
df.dropna(inplace=True)
df.drop(["1 MO"],axis=1,inplace=True)
term = [3/12.,6/12.,1,2,3,5,7,10,20,30]
sample = df.loc["2012-01-01":"2016-12-31"]
sample.tail(5)

,3 MO,6 MO,1 YR,2 YR,3 YR,5 YR,7 YR,10 YR,20 YR,30 YR
Date,,,,,,,,,,
2016-12-23,0.52,0.65,0.87,1.22,1.54,2.04,2.35,2.55,2.86,3.12
2016-12-27,0.51,0.66,0.89,1.28,1.58,2.07,2.37,2.57,2.88,3.14
2016-12-28,0.53,0.62,0.90,1.26,1.55,2.02,2.32,2.51,2.83,3.09
2016-12-29,0.47,0.62,0.85,1.22,1.49,1.96,2.30,2.49,2.82,3.08
2016-12-30,0.51,0.62,0.85,1.20,1.47,1.93,2.25,2.45,2.79,3.06


## Model Construction
Implement DNS Model and find the optimal lamda

In [3]:
# Nelson-Siegel Model (DNS)
class Model:
    def __init__(self,lamda,data,term):
        self.lamda = lamda
        self.data = data
        self.term = term
        self.loadings = self.cal_loading()
        self.betas = []
    
    #Calculate the loading matrix for a specific lamda
    def cal_loading(self):
        loading1 = np.vstack([1]*len(self.term))
        loading2 = np.vstack((1-np.exp(-self.lamda*tau)) / (self.lamda*tau) for tau in self.term)
        loading3 = np.vstack((1-np.exp(-self.lamda*tau)) / (self.lamda*tau) - np.exp(-self.lamda*tau) for tau in self.term)
        loadings = np.hstack([loading1,loading2,loading3])
        return loadings
    
    # Calculate the residuals in time series and cross-sectional for a specific lamda
    def residuals(self,lamda):
        self.update_params(lamda)
        allresiduals = np.empty([len(self.term)])
        # Time-series loop
        for index in self.data.index:
            y = self.data.loc[index].values
            model = sm.OLS(y,self.loadings)
            results = model.fit()
            y_hat = results.predict(self.loadings)
            betas = results.params
            self.betas.append(betas)
            # cross-sectional residuals
            residuals = y - y_hat
            allresiduals = np.concatenate([allresiduals,residuals])
        return allresiduals
    
    # Fit the model to find the optimal lamda
    def fit(self,lamda0,solver='lm'):
        return least_squares(self.residuals,lamda0,method=solver)
    
    # Update the parameters -- lamda and loading matrix
    def update_params(self,lamda):
        self.lamda = lamda
        self.betas = []
        self.loadings = self.cal_loading()
        

In [4]:
# Initial lamda
lamda = 0.6
model = Model(lamda,sample,term)
fo = model.fit(lamda)
print("The optimal lamda is %0.2f"%fo.x)

The optimal lamda is 0.45


## Model Prediction
Use DNS model to forecast and compare the results with those predicted by random walk model

### Step 1 Find the 20 days with largest RMSE

In [5]:
# Construct a dataframe of RMSEs and betas
residuals = model.residuals(model.lamda)
residuals = residuals.reshape(-1,len(term))
residuals = residuals[1:,:]
RMSE = np.sqrt(np.mean(np.power(residuals,2),axis=1))
RMSE = RMSE.reshape(-1,1)
betas = np.vstack(model.betas)
results_df = pd.DataFrame(np.hstack([RMSE,betas]),sample.index,columns=["RMSE","beta1","beta2","beta3"])
results_df.tail(5)

,RMSE,beta1,beta2,beta3
Date,,,,
2016-12-23,0.039259,3.319071,-2.938875,-0.522285
2016-12-27,0.036175,3.319887,-2.944913,-0.382847
2016-12-28,0.037766,3.264882,-2.879944,-0.433325
2016-12-29,0.037340,3.272291,-2.915716,-0.559228
2016-12-30,0.036516,3.253188,-2.860177,-0.699686


In [6]:
results_df_sorted = results_df.sort_values(by="RMSE",ascending=False)
index_best20 = results_df_sorted.iloc[:20].index
print("Followings are the 20 days with largest RMSE")
index_best20

Followings are the 20 days with largest RMSE


DatetimeIndex(['2013-12-31', '2013-12-24', '2013-12-27', '2014-01-02',
               '2013-12-30', '2014-01-03', '2013-12-23', '2013-12-26',
               '2013-12-19', '2014-04-21', '2014-04-24', '2013-12-20',
               '2014-01-08', '2014-04-17', '2014-01-06', '2014-04-22',
               '2014-04-28', '2014-04-25', '2014-04-23', '2014-04-03'],
              dtype='datetime64[ns]', name='Date', freq=None)

### Step 2 Use DNS Model to predict
1. Use the 6 month data prior to each of those 20 days as sample; 
2. Do AR fitting and prediction for betas
3. Calculate the predicted CMT rates by plugging predicted betas into DNS model
4. Get the summary of prediction results

In [7]:
# Define a function to find the next business day
HOLIDAYS_US = holidays.US()
def next_business_day(startDate,gap):
    next_day = startDate+timedelta(days=gap)
    while next_day.weekday() in holidays.WEEKEND or next_day in HOLIDAYS_US:
        next_day += timedelta(days=1)
    return next_day
# Define a function to compute half-lives
def get_halflife(y):
    y_lag = y.shift(1)
    y_ret = y - y_lag
    y_lag = y_lag[1:]
    y_ret = y_ret[1:]
    y_lag2 = sm.add_constant(y_lag)

    model = sm.OLS(y_ret.values,y_lag2.values)
    res = model.fit()

    halflife = round(-np.log(2) / res.params[1],0)
    return halflife


In [8]:
# Do 3 predictions for t+5d, t+10d, t+1M
loadings = model.loadings
maturity = sample.columns.values
length_sample = 182
length_predict = [5,10,20]
results_5d = []
results_10d = []
results_1M = []

hls_beta1 = []
hls_beta2 = []
hls_beta3 = []

series_num = 1
for index in index_best20:
    # For each of date in best20, select data 6 month prior to that date as sample 
    # to fit AR models for betas
    endDate_sample = index
    startDate_sample = next_business_day(endDate_sample,-length_sample)
    sample_6m = results_df.loc[startDate_sample:endDate_sample,["beta1","beta2","beta3"]]
    
    # Calculate the HL of betas in sample
    hl_beta1 = get_halflife(sample_6m["beta1"])
    hl_beta2 = get_halflife(sample_6m["beta2"])
    hl_beta3 = get_halflife(sample_6m["beta3"])
    hls_beta1.append(hl_beta1)
    hls_beta2.append(hl_beta2)
    hls_beta3.append(hl_beta3)
    
    # Calculate the index for prediction
    start_index = len(sample_6m)
    end_index = start_index + 20
    # Fit the AR models for betas and predict
    AR_beta1 = ARMA(sample_6m["beta1"].values,(1,0))
    result1 = AR_beta1.fit()
    beta1_hat = result1.predict(start=start_index,end=end_index,dynamic=True)

    AR_beta2 = ARMA(sample_6m["beta2"].values,(1,0))
    result2 = AR_beta2.fit()
    beta2_hat = result2.predict(start=start_index,end=end_index,dynamic=True)

    AR_beta3 = ARMA(sample_6m["beta3"].values,(1,0))
    result3 = AR_beta3.fit()
    beta3_hat = result3.predict(start=start_index,end=end_index,dynamic=True)

    predictions = []
    for length in length_predict:
        # Forecast the CMT rate based on loadings and predicted betas
        CMT_hat = beta1_hat[length-1]*loadings[:,0]+beta2_hat[length-1]*loadings[:,1]+beta3_hat[length-1]*loadings[:,2]

        # Slice the corresponding observation, and calculate the residuals
        CMT = sample.loc[next_business_day(endDate_sample,length)].values
        residuals = CMT - CMT_hat

        series = np.array([series_num]*len(CMT))

        data = np.hstack([maturity.reshape(-1,1),series.reshape(-1,1),CMT_hat.reshape(-1,1),residuals.reshape(-1,1)])

        df_predict = pd.DataFrame(data,columns=["Maturity","Series","prediction","residual"])

        predictions.append(df_predict)
    
    results_5d.append(predictions[0])
    results_10d.append(predictions[1])
    results_1M.append(predictions[2])
    
    series_num += 1

hl_df = pd.DataFrame({"beta1":hls_beta1,"beta2":hls_beta2,"beta3":hls_beta3},index=index_best20)
hl_df.head(5)

,beta1,beta2,beta3
Date,,,
2013-12-31,8.0,8.0,16.0
2013-12-24,10.0,9.0,16.0
2013-12-27,9.0,9.0,16.0
2014-01-02,11.0,11.0,14.0
2013-12-30,8.0,8.0,16.0


In [9]:
results_5d = pd.concat(results_5d)
results_10d = pd.concat(results_10d)
results_1M = pd.concat(results_1M)
results_5d.head(10)

,Maturity,Series,prediction,residual
0,3 MO,1,-0.00607123,0.0560712
1,6 MO,1,0.0515439,0.0284561
2,1 YR,1,0.194281,-0.0742805
3,2 YR,1,0.544726,-0.144726
4,3 YR,1,0.924423,-0.144423
5,5 YR,1,1.63469,0.0653145
6,7 YR,1,2.20727,0.17273
7,10 YR,1,2.81755,0.162453
8,20 YR,1,3.72293,-0.0629295
9,30 YR,1,4.04522,-0.14522


In [10]:
# Convert the predicted results to a statistic summary
def df_convert(df):
    means = []
    stds = []
    RMSEs = []
    maturity = df["Maturity"].values[:len(term)]
    for i in maturity:
        grouped = df.groupby("Maturity")
        grouped_i = grouped.get_group(i)
        mean = np.mean(grouped_i["prediction"])
        std = np.std(grouped_i["prediction"])
        RMSE = np.sqrt(np.mean(np.power(grouped_i["residual"],2)))
        means.append(mean)
        stds.append(std)
        RMSEs.append(RMSE)
    result = pd.DataFrame({"Mean":means,"Std":stds,"RMSE":RMSEs},index=maturity)
    return result

In [11]:
final_5d = df_convert(results_5d)
final_10d = df_convert(results_10d)
final_1M = df_convert(results_1M)
finals_DNS = pd.concat([final_5d,final_10d,final_1M],axis=1)
columns = ['Mean_5d', 'RMSE_5d', 'Std_5d','Mean_10d', 'RMSE_10d', 'Std_10d','Mean_1M', 'RMSE_1M', 'Std_1M']
finals_DNS.columns = columns
finals_DNS

,Mean_5d,RMSE_5d,Std_5d,Mean_10d,RMSE_10d,Std_10d,Mean_1M,RMSE_1M,Std_1M
3 MO,-0.047015,0.098288,0.047202,-0.041555,0.094988,0.053845,-0.034394,0.088234,0.057673
6 MO,0.027894,0.045577,0.025987,0.030217,0.043029,0.031378,0.032732,0.039158,0.034382
1 YR,0.195569,0.080534,0.012401,0.192716,0.081237,0.014262,0.187640,0.083000,0.018875
2 YR,0.567950,0.160405,0.042714,0.558090,0.158277,0.046749,0.542970,0.153928,0.054782
3 YR,0.946759,0.125014,0.049469,0.932882,0.119568,0.056847,0.912274,0.113781,0.068988
5 YR,1.626309,0.101277,0.034195,1.609132,0.098187,0.043021,1.584612,0.113363,0.059803
7 YR,2.159538,0.194752,0.037504,2.141817,0.180609,0.031031,2.117337,0.172590,0.038764
10 YR,2.719648,0.165098,0.078086,2.702540,0.164543,0.058686,2.679901,0.152380,0.036683
20 YR,3.542224,0.098838,0.159437,3.527240,0.147169,0.133223,3.509166,0.182105,0.096286
30 YR,3.834177,0.126285,0.190173,3.820073,0.166075,0.162069,3.803813,0.197906,0.122024


### Step 3 Use Ramdom walk to predict

In [12]:
# Random Walk Model
results_5d_RW = []
results_10d_RW = []
results_1M_RW = []

series_num = 1
for index in index_best20:
    # For each of date in best20, select data 6 month prior to that date as sample 
    # to fit AR models for betas
    endDate_sample = index
    predictions = []
    
    for length in length_predict:
        date_lag = next_business_day(endDate_sample,length-1)
        date_observation = next_business_day(endDate_sample,length)

        CMT_hat = sample.loc[date_lag].values
        CMT = sample.loc[date_observation].values
        
        residuals = CMT - CMT_hat

        series = np.array([series_num]*len(CMT))
        
        data = np.hstack([maturity.reshape(-1,1),series.reshape(-1,1),CMT_hat.reshape(-1,1),residuals.reshape(-1,1)])

        df_predict = pd.DataFrame(data,columns=["Maturity","Series","prediction","residual"])

        predictions.append(df_predict)
    
    results_5d_RW.append(predictions[0])
    results_10d_RW.append(predictions[1])
    results_1M_RW.append(predictions[2])
    
    series_num += 1
    
results_5d_RW = pd.concat(results_5d_RW)
results_10d_RW = pd.concat(results_10d_RW)
results_1M_RW = pd.concat(results_1M_RW)    
results_5d_RW.head(10)

,Maturity,Series,prediction,residual
0,3 MO,1,0.05,0
1,6 MO,1,0.08,0
2,1 YR,1,0.12,0
3,2 YR,1,0.4,0
4,3 YR,1,0.78,0
5,5 YR,1,1.7,0
6,7 YR,1,2.38,0
7,10 YR,1,2.98,0
8,20 YR,1,3.66,0
9,30 YR,1,3.9,0


In [13]:
final_5d_RW = df_convert(results_5d_RW)
final_10d_RW = df_convert(results_10d_RW)
final_1M_RW = df_convert(results_1M_RW)
finals_RW = pd.concat([final_5d_RW,final_10d_RW,final_1M_RW],axis=1)
columns = ['Mean_5d_RW', 'RMSE_5d_RW', 'Std_5d_RW','Mean_10d_RW', 'RMSE_10d_RW', 'Std_10d','Mean_1M_RW', 'RMSE_1M_RW', 'Std_1M_RW']
finals_RW.columns = columns
finals_RW

,Mean_5d_RW,RMSE_5d_RW,Std_5d_RW,Mean_10d_RW,RMSE_10d_RW,Std_10d,Mean_1M_RW,RMSE_1M_RW,Std_1M_RW
3 MO,0.0470,0.008367,0.018735,0.0430,0.005000,0.015843,0.0365,0.003873,0.007263
6 MO,0.0720,0.005916,0.020396,0.0670,0.005477,0.017349,0.0615,0.007071,0.010137
1 YR,0.1180,0.006325,0.012884,0.1155,0.006325,0.012031,0.1090,0.007416,0.013000
2 YR,0.4075,0.015000,0.020946,0.4075,0.013038,0.019203,0.4020,0.018574,0.022494
3 YR,0.8265,0.023238,0.059856,0.8230,0.024799,0.057541,0.8220,0.029411,0.053254
5 YR,1.7060,0.030249,0.036797,1.6835,0.027749,0.045747,1.6570,0.037550,0.054690
7 YR,2.3400,0.032016,0.070143,2.3040,0.030984,0.088736,2.2560,0.038987,0.083331
10 YR,2.8575,0.028636,0.144183,2.8235,0.027386,0.158470,2.7710,0.033912,0.133563
20 YR,3.4770,0.030000,0.210002,3.4390,0.030984,0.223515,3.3850,0.033015,0.184377
30 YR,3.7215,0.028196,0.207949,3.6870,0.029917,0.218589,3.6400,0.030414,0.170822


### DNS v.s Random Walk
Since its corresponding RMSEs are larger, the DNS model performs no better than random walk